<a href="https://colab.research.google.com/github/chernoskutov1989dv/Spark-Project-Car-database/blob/master/_google_collab_pyspark_d_chernoskutov_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка через pip

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 41.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=731a5e7d74920ed14bc4291d49249c7e9f637ec6e193057ad9a8995a0eac30d0
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [ ]:
spark.version

'3.3.0'

In [ ]:
!head -h5 data/cars.csv

head: invalid option -- 'h'
Try 'head --help' for more information.


# Установка из исходников



Чуть более сложная установка и настройка, однако более гибкая (можно выбрать любую версию Spark и Java)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

tar: spark-3.1.2-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark.version

'3.3.0'

# Дополнительные фишки Google Colab

In [ ]:
# Загрузка файла с локального диска

from google.colab import files
files.upload()

{}

In [ ]:
# примонтировать Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spark.read.format ("csv").load ("datacars.csv")

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string]

In [ ]:
spark.read.format ("csv").option("header", "true").load ("datacars.csv")

DataFrame[manufacturer_name: string, model_name: string, transmission: string, color: string, odometer_value: string, year_produced: string, engine_fuel: string, engine_has_gas: string, engine_type: string, engine_capacity: string, body_type: string, has_warranty: string, state: string, drivetrain: string, price_usd: string, is_exchangeable: string, location_region: string, number_of_photos: string, up_counter: string, feature_0: string, feature_1: string, feature_2: string, feature_3: string, feature_4: string, feature_5: string, feature_6: string, feature_7: string, feature_8: string, feature_9: string, duration_listed: string]

In [ ]:
df = spark.read.format ("csv").option("header", "true").load ("datacars.csv")

In [ ]:
df.select ("manufacturer_name", "model_name").show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [ ]:
import pyspark.sql.functions as F

df.select (F.col ("manufacturer_name"), F.col("model_name")).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [ ]:
df.select("manufacturer_name", "model_name", "transmission").filter("manufacturer_name = 'Audi'").show(5)




+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|        Q7|   automatic|
|             Audi|        TT|   automatic|
|             Audi|       100|  mechanical|
|             Audi|        A6|   automatic|
|             Audi|        Q3|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [ ]:
(
    df
    .select("manufacturer_name", "model_name", "transmission")
    .filter("manufacturer_name = 'Audi' and transmission = 'mechanical'")
    .show(5)
)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



In [ ]:
NAME = 'Audi'   #Если параметр приходит из ВНЕ
(
   df
   .select ("manufacturer_name", "model_name", "transmission")  
   .filter(F.col("manufacturer_name") == NAME)
   .filter(F.col("transmission") =="mechanical")
   .show(5)

)


+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



In [ ]:
(
    df
    .select("manufacturer_name", "model_name", "transmission")
    .filter(F.col("manufacturer_name") == "Audi")
    .filter(F.col("transmission") != "mechanical")
    .show(5)
)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|        Q7|   automatic|
|             Audi|        TT|   automatic|
|             Audi|        A6|   automatic|
|             Audi|        Q3|   automatic|
|             Audi|        Q5|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



# Метод подсчета всех строк:

In [ ]:
df.count()

38531

# Метод подсчета уникальных строк:

In [ ]:
df.distinct().count()


38491

# Метод подсчета уникальных строк по конкретной колонке, которая точно повторяется в базе:

In [ ]:
df.select("manufacturer_name").distinct().count()

55

# Группирование и агреация


# сгруппировать по  manufacturer_name и посчитать количество

In [ ]:
df.groupBy("manufacturer_name").count() 

DataFrame[manufacturer_name: string, count: bigint]

In [ ]:
df.groupBy("manufacturer_name").count().show()

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
|             Jeep|  107|
|       Mitsubishi|  887|
|              ГАЗ|  200|
|              Kia|  912|
|             Mini|   68|
|        Chevrolet|  436|
|            Volvo|  721|
|            Lifan|   47|
|          Hyundai| 1116|
|             LADA|  146|
|        SsangYong|   79|
|             Audi| 2468|
|             Seat|  303|
|         Cadillac|   43|
|          Pontiac|   42|
+-----------------+-----+
only showing top 20 rows



# сгруппировать по возрастанию 

In [ ]:
df.groupBy ("manufacturer_name").count().orderBy ("count").show(5) 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Great Wall|   36|
|          Lincoln|   36|
|              ЗАЗ|   42|
|          Pontiac|   42|
|         Cadillac|   43|
+-----------------+-----+
only showing top 5 rows



# сгруппировать по убыванию с колонкой col(): и модификатором desk ():

In [ ]:
df.groupBy ("manufacturer_name").count().orderBy(F.col("count").desc()).show(5) 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



# сгруппировать по убыванию альтернативно (без desc)

In [ ]:
df.groupBy ("manufacturer_name").count().orderBy(-F.col("count")).show(5) 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



# Переименовать существующую колонку 

In [ ]:
df.withColumn ("next_year", F.col ("year_produced")+1).select ("year_produced", "next_year").show(5)

+-------------+---------+
|year_produced|next_year|
+-------------+---------+
|         2010|   2011.0|
|         2002|   2003.0|
|         2001|   2002.0|
|         1999|   2000.0|
|         2001|   2002.0|
+-------------+---------+
only showing top 5 rows



#Вывести схему дата фрейма

In [ ]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+-----------------+---------+------------+---------+----------+-----------------+---------------+----------------+-----------------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+
|summary|manufacturer_name|        model_name|transmission| color|    odometer_value|     year_produced|  engine_fuel|engine_has_gas|engine_type|  engine_capacity|body_type|has_warranty|    state|drivetrain|        price_usd|is_exchangeable| location_region| number_of_photos|        up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|   duration_listed|
+-------+-----------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+-----------------+-------

In [ ]:
df.select ("manufacturer_name", "year_produced").describe().show()

+-------+-----------------+------------------+
|summary|manufacturer_name|     year_produced|
+-------+-----------------+------------------+
|  count|            38531|             38531|
|   mean|             null|2002.9437336170874|
| stddev|             null| 8.065730511309935|
|    min|            Acura|              1942|
|    max|              УАЗ|              2019|
+-------+-----------------+------------------+



Остановка сессии  Spark

In [ ]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

def main ():

    spark =  SparkSession.builder.getOrCreate()
    df = spark.read.format ("csv").option ("header", "true").load("datacars.csv")
    
      output = (
        df
        .groupBy("manufacture_name")
        .agg(
            F.count("manufacture_name").alias("count"),
            F.round (F.avg("year_produced")).alias("avg_year"),
            F.min("price_usd").alias("min_price"),
            F.max ("price_usd").alias("max_price")
        )

    )
    output.show(6)
    
main()






In [ ]:
    output = (
        df
        .groupBy("manufacture_name")
        .agg(
            F.count("manufacture_name").alias("count"),
            F.round (F.avg("year_produced")).alias("avg_year"),
            F.min("price_usd").alias("min_price"),
            F.max ("price_usd").alias("max_price")
        )

    )
    output.show(6)
main()

AnalysisException: ignored